In [2]:
# Creating Date List For Backtesting
import sys
sys.path.append(r'C:\Users\kocat_000\quantFinance\PycharmProjects')
import backtesting.utilities as bu
dates_output = bu.get_backtesting_dates(date_to=20151228,years_back=10)
double_dates = sorted(dates_output['double_dates'],reverse=False)
selected_dates = double_dates
selected_dates


[20060104,
 20060111,
 20060118,
 20060125,
 20060201,
 20060208,
 20060215,
 20060222,
 20060301,
 20060308,
 20060315,
 20060322,
 20060329,
 20060405,
 20060412,
 20060419,
 20060426,
 20060503,
 20060510,
 20060517,
 20060524,
 20060531,
 20060607,
 20060614,
 20060621,
 20060628,
 20060705,
 20060712,
 20060719,
 20060726,
 20060802,
 20060809,
 20060816,
 20060823,
 20060830,
 20060906,
 20060913,
 20060920,
 20060927,
 20061004,
 20061011,
 20061018,
 20061025,
 20061101,
 20061108,
 20061115,
 20061122,
 20061129,
 20061206,
 20061213,
 20061220,
 20061227,
 20070103,
 20070110,
 20070117,
 20070124,
 20070131,
 20070207,
 20070214,
 20070221,
 20070228,
 20070307,
 20070314,
 20070321,
 20070328,
 20070404,
 20070411,
 20070418,
 20070425,
 20070502,
 20070509,
 20070516,
 20070523,
 20070530,
 20070606,
 20070613,
 20070620,
 20070627,
 20070711,
 20070718,
 20070725,
 20070801,
 20070808,
 20070815,
 20070822,
 20070829,
 20070905,
 20070912,
 20070919,
 20070926,
 20071003,

In [3]:
# Historical Butterfly Report Run:

import opportunity_constructs.futures_butterfly as fb

for date_to in selected_dates:
    fb.generate_futures_butterfly_sheet_4date(date_to=date_to)
    

    

In [3]:
import backtesting.backtest_futures_butterfly as bfb
import get_price.get_futures_price as gfp
import contract_utilities.contract_meta_info as cmi
import shared.directory_names as dn
import pickle

butterfly_output = bfb.backtest_futures_butterfly(date_list = selected_dates,
                                                  use_existing_filesQ=True)

big_data = butterfly_output['big_data']
backtest_output = butterfly_output['backtest_output']


In [5]:
# Basic Stats

import backtesting.backtest_stats as bs
import shared.statistics as stats
import numpy as np
import pandas as pd

standard_short_stats1 = bs.get_summary_stats(big_data[(big_data['Q']>85) & (big_data['QF']>60)]['pnl_final'].values)
standard_long_stats1 = bs.get_summary_stats(big_data[(big_data['Q']<15) & (big_data['QF']<40)]['pnl_final'].values)

standard_short_stats2 = bs.get_summary_stats(big_data[(big_data['Q']>85) & (big_data['QF']>60)]['pnl_final'].values)
standard_long_stats2 = bs.get_summary_stats(big_data[(big_data['Q']<15) & (big_data['QF']<40)]['pnl_final'].values)

summary_table = pd.DataFrame({'class': ['Short1', 'Short2', 'Long1', 'Long2'],
              'reward_risk': [standard_short_stats1['reward_risk'],
                              standard_short_stats2['reward_risk'],
                              standard_long_stats1['reward_risk'],
                              standard_long_stats2['reward_risk']],
              'mean_pnl': [standard_short_stats1['mean_pnl'],
                              standard_short_stats2['mean_pnl'],
                              standard_long_stats1['mean_pnl'],
                              standard_long_stats2['mean_pnl']],
              'total_pnl': [standard_short_stats1['total_pnl'],
                              standard_short_stats2['total_pnl'],
                              standard_long_stats1['total_pnl'],
                              standard_long_stats2['total_pnl']]})

summary_table[['class','reward_risk','mean_pnl','total_pnl']]



,class,reward_risk,mean_pnl,total_pnl
0,Short1,0.335810,970.909900,16491875.569
1,Short2,0.335810,970.909900,16491875.569
2,Long1,0.309032,838.466531,16291404.698
3,Long2,0.309032,838.466531,16291404.698


In [54]:


import backtesting.utilities as bu
num_buckets = 10
import backtesting.backtest_stats as bs
import pandas as pd
import shared.statistics as stats
indicator = 'Q'

bucket_limits = bu.get_equal_length_bucket_limits(min_value=0,
                                                  max_value=100,
                                                  num_buckets=num_buckets)

bucket_limits = stats.get_number_from_quantile(y=big_data[indicator].values,quantile_list=quantile_limits)

mean_pnl_list = []
mean_indicator_list = []
reward_risk_list = []



for i in range(num_buckets):
    
    if i==0:
        bucket_data = big_data.loc[big_data[indicator]<=bucket_limits[i],['pnl_long5','pnl_short5',indicator]]
    elif i<num_buckets-1:
        bucket_data = big_data.loc[(big_data[indicator]>bucket_limits[i-1])&
                                   (big_data[indicator]<=bucket_limits[i]),['pnl_long5','pnl_short5',indicator]]
    else:
        bucket_data = big_data.loc[big_data[indicator]>bucket_limits[i-1],['pnl_long5','pnl_short5',indicator]]
    
    if i<=(num_buckets/2):
        signed_pnl = bucket_data['pnl_long5']
    else:
        signed_pnl = bucket_data['pnl_short5']
        
    stats_output = bs.get_summary_stats(signed_pnl.values)    
    mean_pnl_list.append(stats_output['mean_pnl'])
    mean_indicator_list.append(bucket_data[indicator].mean())
    reward_risk_list.append(stats_output['reward_risk'])
        

pd.DataFrame.from_items([('indicator', mean_indicator_list),
                         ('mean_pnl', mean_pnl_list),
                         ('reard_risk',reward_risk_list)])




,indicator,mean_pnl,reard_risk
0,4.546382,869.659724,0.311450
1,13.948948,27.157795,0.009271
2,24.014070,33.085709,0.011431
3,34.031604,-36.202453,-0.013095
4,43.484630,-81.175523,-0.029673
5,53.402439,-89.004819,-0.031629
6,63.908375,65271.429349,23.982742
7,75.010449,278.593746,0.095447
8,86.079809,53921.808711,17.433780
9,95.857684,960.199669,0.324820


In [58]:
big_data[big_data['pnl_long5']<-10000000]

,ticker1,ticker2,ticker3,tickerHead,tickerClass,trDte1,trDte2,trDte3,multiplier,agg,...,price2,price3,pnl_long5,pnl_short5,pnl_long2,pnl_short2,pnl_long1,pnl_short1,report_date,pnl_final
168,EDZ2018,EDH2019,EDM2019,ED,STIR,1225,1288,1351,2500,3,...,96.135,96.005,-1106259198,1.116735e+09,-1106259198,1.116735e+09,-1106259198,1.116735e+09,20140219,1.116735e+09
190,EDM2018,EDZ2018,EDM2019,ED,STIR,1098,1225,1351,2500,3,...,96.275,96.005,-550540314,5.632843e+08,-550540314,5.632843e+08,-550540314,5.632843e+08,20140219,5.632843e+08
210,EDZ2017,EDU2018,EDM2019,ED,STIR,972,1161,1351,2500,3,...,96.435,96.005,-348362343,3.677158e+08,-73367343,7.744331e+07,-348360768,3.677141e+08,20140219,3.677158e+08


In [56]:
import matplotlib.pyplot as plt
plt.plot(big_data['pnl_long5'])
plt.grid()
plt.show()

In [48]:
quantile_limits = bu.get_equal_length_bucket_limits(min_value=0,
                                                  max_value=100,
                                                  num_buckets=num_buckets)
import shared.statistics as stats
stats.get_number_from_quantile(y=big_data[indicator].values,quantile_list=quantile_limits)

array([-1.16, -0.73, -0.44, -0.2 ,  0.  ,  0.18,  0.4 ,  0.7 ,  1.17])

In [ ]:
# Z1 seems to be a good thing to look in general


standard_short_stats = bs.get_summary_stats(big_data[(big_data['Q']>85) & 
                                                      (big_data['QF']>60)]['pnl_final'].values)

standard_long_stats = bs.get_summary_stats(big_data[(big_data['Q']<15) & 
                                                     (big_data['QF']<40)]['pnl_final'].values)


standard_short_stats2 = bs.get_summary_stats(big_data[(big_data['Q']>85) & 
                                                      (big_data['QF']>60) &
                                                      (big_data['z1']>1)]['pnl_final'].values)

standard_long_stats2 = bs.get_summary_stats(big_data[(big_data['Q']<15) & 
                                                     (big_data['QF']<40) &
                                                      (big_data['z1']<-1)
                                                    ]['pnl_final'].values)


summary_table = pd.DataFrame({'class': ['Short1', 'Short2', 'Long1', 'Long2'],
              'reward_risk': [standard_short_stats1['reward_risk'],
                              standard_short_stats2['reward_risk'],
                              standard_long_stats1['reward_risk'],
                              standard_long_stats2['reward_risk']],
              'mean_pnl': [standard_short_stats1['mean_pnl'],
                              standard_short_stats2['mean_pnl'],
                              standard_long_stats1['mean_pnl'],
                              standard_long_stats2['mean_pnl']],
              'total_pnl': [standard_short_stats1['total_pnl'],
                              standard_short_stats2['total_pnl'],
                              standard_long_stats1['total_pnl'],
                              standard_long_stats2['total_pnl']]})

summary_table[['class','reward_risk','mean_pnl','total_pnl']]


In [ ]:
# Z2 seems to be a good thing to look at for long butterflies


standard_short_stats = bs.get_summary_stats(big_data[(big_data['Q']>85) & 
                                                      (big_data['QF']>60)]['pnl_final'].values)

standard_long_stats = bs.get_summary_stats(big_data[(big_data['Q']<15) & 
                                                     (big_data['QF']<40)]['pnl_final'].values)


standard_short_stats2 = bs.get_summary_stats(big_data[(big_data['Q']>85) & 
                                                      (big_data['QF']>60) &
                                                      (big_data['z2']>1)]['pnl_final'].values)

standard_long_stats2 = bs.get_summary_stats(big_data[(big_data['Q']<15) & 
                                                     (big_data['QF']<40) &
                                                      (big_data['z2']<-1)
                                                    ]['pnl_final'].values)


summary_table = pd.DataFrame({'class': ['Short1', 'Short2', 'Long1', 'Long2'],
              'reward_risk': [standard_short_stats1['reward_risk'],
                              standard_short_stats2['reward_risk'],
                              standard_long_stats1['reward_risk'],
                              standard_long_stats2['reward_risk']],
              'mean_pnl': [standard_short_stats1['mean_pnl'],
                              standard_short_stats2['mean_pnl'],
                              standard_long_stats1['mean_pnl'],
                              standard_long_stats2['mean_pnl']],
              'total_pnl': [standard_short_stats1['total_pnl'],
                              standard_short_stats2['total_pnl'],
                              standard_long_stats1['total_pnl'],
                              standard_long_stats2['total_pnl']]})

summary_table[['class','reward_risk','mean_pnl','total_pnl']]


In [ ]:
# R1 seems to be a good thing to look at in general 

standard_short_stats = bs.get_summary_stats(big_data[(big_data['Q']>85) & 
                                                      (big_data['QF']>60)]['pnl_final'].values)

standard_long_stats = bs.get_summary_stats(big_data[(big_data['Q']<15) & 
                                                     (big_data['QF']<40)]['pnl_final'].values)


standard_short_stats2 = bs.get_summary_stats(big_data[(big_data['Q']>85) & 
                                                      (big_data['QF']>60) &
                                                      (big_data['r1']>50)]['pnl_final'].values)

standard_long_stats2 = bs.get_summary_stats(big_data[(big_data['Q']<15) & 
                                                     (big_data['QF']<40) &
                                                      (big_data['r1']>50)
                                                    ]['pnl_final'].values)


summary_table = pd.DataFrame({'class': ['Short1', 'Short2', 'Long1', 'Long2'],
              'reward_risk': [standard_short_stats1['reward_risk'],
                              standard_short_stats2['reward_risk'],
                              standard_long_stats1['reward_risk'],
                              standard_long_stats2['reward_risk']],
              'mean_pnl': [standard_short_stats1['mean_pnl'],
                              standard_short_stats2['mean_pnl'],
                              standard_long_stats1['mean_pnl'],
                              standard_long_stats2['mean_pnl']],
              'total_pnl': [standard_short_stats1['total_pnl'],
                              standard_short_stats2['total_pnl'],
                              standard_long_stats1['total_pnl'],
                              standard_long_stats2['total_pnl']]})

summary_table[['class','reward_risk','mean_pnl','total_pnl']]


In [ ]:
# R2 seems to be a good thing to look at in general 

standard_short_stats = bs.get_summary_stats(big_data[(big_data['Q']>85) & 
                                                      (big_data['QF']>60)]['pnl_final'].values)

standard_long_stats = bs.get_summary_stats(big_data[(big_data['Q']<15) & 
                                                     (big_data['QF']<40)]['pnl_final'].values)


standard_short_stats2 = bs.get_summary_stats(big_data[(big_data['Q']>85) & 
                                                      (big_data['QF']>60) &
                                                      (big_data['r2']>50)]['pnl_final'].values)

standard_long_stats2 = bs.get_summary_stats(big_data[(big_data['Q']<15) & 
                                                     (big_data['QF']<40) &
                                                      (big_data['r2']>50)
                                                    ]['pnl_final'].values)


summary_table = pd.DataFrame({'class': ['Short1', 'Short2', 'Long1', 'Long2'],
              'reward_risk': [standard_short_stats1['reward_risk'],
                              standard_short_stats2['reward_risk'],
                              standard_long_stats1['reward_risk'],
                              standard_long_stats2['reward_risk']],
              'mean_pnl': [standard_short_stats1['mean_pnl'],
                              standard_short_stats2['mean_pnl'],
                              standard_long_stats1['mean_pnl'],
                              standard_long_stats2['mean_pnl']],
              'total_pnl': [standard_short_stats1['total_pnl'],
                              standard_short_stats2['total_pnl'],
                              standard_long_stats1['total_pnl'],
                              standard_long_stats2['total_pnl']]})

summary_table[['class','reward_risk','mean_pnl','total_pnl']]

In [ ]:
# Recent Vol Ratio Does Seem to Have Certain Implications For Sizing

standard_short_stats = bs.get_summary_stats(big_data[(big_data['Q']>85) & 
                                                      (big_data['QF']>60)]['pnl_final'].values)

standard_long_stats = bs.get_summary_stats(big_data[(big_data['Q']<15) & 
                                                     (big_data['QF']<40)]['pnl_final'].values)


standard_short_stats2 = bs.get_summary_stats(big_data[(big_data['Q']>85) & 
                                                      (big_data['QF']>60) &
                                                      (big_data['recent_vol_ratio']>1)]['pnl_final'].values)

standard_long_stats2 = bs.get_summary_stats(big_data[(big_data['Q']<15) & 
                                                     (big_data['QF']<40) &
                                                      (big_data['recent_vol_ratio']>1)
                                                    ]['pnl_final'].values)


summary_table = pd.DataFrame({'class': ['Short1', 'Short2', 'Long1', 'Long2'],
              'reward_risk': [standard_short_stats1['reward_risk'],
                              standard_short_stats2['reward_risk'],
                              standard_long_stats1['reward_risk'],
                              standard_long_stats2['reward_risk']],
              'mean_pnl': [standard_short_stats1['mean_pnl'],
                              standard_short_stats2['mean_pnl'],
                              standard_long_stats1['mean_pnl'],
                              standard_long_stats2['mean_pnl']],
                'downside20': [standard_short_stats1['downside20'],
                              standard_short_stats2['downside20'],
                              standard_long_stats1['downside20'],
                              standard_long_stats2['downside20']],                  
              'total_pnl': [standard_short_stats1['total_pnl'],
                              standard_short_stats2['total_pnl'],
                              standard_long_stats1['total_pnl'],
                              standard_long_stats2['total_pnl']]})

summary_table[['class','reward_risk','mean_pnl','downside20','total_pnl']]

In [ ]:
# Recent Vol Ratio Does Seem to Have Certain Implications For Sizing

standard_short_stats = bs.get_summary_stats(big_data[(big_data['Q']>85) & 
                                                      (big_data['QF']>60)]['pnl_final'].values)

standard_long_stats = bs.get_summary_stats(big_data[(big_data['Q']<15) & 
                                                     (big_data['QF']<40)]['pnl_final'].values)


standard_short_stats2 = bs.get_summary_stats(big_data[(big_data['Q']>85) & 
                                                      (big_data['QF']>60) &
                                                      (big_data['recent_5day_pnl']<2*big_data['upside'])]['pnl_final'].values)

standard_long_stats2 = bs.get_summary_stats(big_data[(big_data['Q']<15) & 
                                                     (big_data['QF']<40) &
                                                      ((big_data['recent_5day_pnl']>2*big_data['downside']))
                                                    ]['pnl_final'].values)


summary_table = pd.DataFrame({'class': ['Short1', 'Short2', 'Long1', 'Long2'],
              'reward_risk': [standard_short_stats1['reward_risk'],
                              standard_short_stats2['reward_risk'],
                              standard_long_stats1['reward_risk'],
                              standard_long_stats2['reward_risk']],
              'mean_pnl': [standard_short_stats1['mean_pnl'],
                              standard_short_stats2['mean_pnl'],
                              standard_long_stats1['mean_pnl'],
                              standard_long_stats2['mean_pnl']],
                'downside20': [standard_short_stats1['downside20'],
                              standard_short_stats2['downside20'],
                              standard_long_stats1['downside20'],
                              standard_long_stats2['downside20']],                  
              'total_pnl': [standard_short_stats1['total_pnl'],
                              standard_short_stats2['total_pnl'],
                              standard_long_stats1['total_pnl'],
                              standard_long_stats2['total_pnl']]})

summary_table[['class','reward_risk','mean_pnl','downside20','total_pnl']]

In [ ]:
standard_short_stats = bs.get_summary_stats(big_data[(big_data['Q']>85) & 
                                                      (big_data['QF']>60)]['pnl_final'].values)

standard_long_stats = bs.get_summary_stats(big_data[(big_data['Q']<15) & 
                                                     (big_data['QF']<40)]['pnl_final'].values)



standard_short_stats2 = bs.get_summary_stats(big_data[(big_data['Q']>85) & 
                                                      (big_data['QF']>60) &
                                                      (big_data['r2']>50)&
                                                      (big_data['z2']>1.5)
                                                     ]['pnl_final'].values)

standard_long_stats2 = bs.get_summary_stats(big_data[(big_data['Q']<15) & 
                                                     (big_data['QF']<40) &
                                                      (big_data['r2']>50) &
                                                      (big_data['z2']<-1.5)
                                                    ]['pnl_final'].values)


summary_table = pd.DataFrame({'class': ['Short1', 'Short2', 'Long1', 'Long2'],
              'reward_risk': [standard_short_stats1['reward_risk'],
                              standard_short_stats2['reward_risk'],
                              standard_long_stats1['reward_risk'],
                              standard_long_stats2['reward_risk']],
              'mean_pnl': [standard_short_stats1['mean_pnl'],
                              standard_short_stats2['mean_pnl'],
                              standard_long_stats1['mean_pnl'],
                              standard_long_stats2['mean_pnl']],
              'total_pnl': [standard_short_stats1['total_pnl'],
                              standard_short_stats2['total_pnl'],
                              standard_long_stats1['total_pnl'],
                              standard_long_stats2['total_pnl']]})

summary_table[['class','reward_risk','mean_pnl','total_pnl']]


In [ ]:
# Ticker Head Based Analysis

import contract_utilities.contract_meta_info as cmi
ticker_head_list = cmi.futures_butterfly_strategy_tickerhead_list
import pandas as pd
import backtesting.backtest_stats as bs

ticker_head_summary_dataframe = pd.DataFrame()
ticker_head_summary_dataframe['ticker_head'] = ticker_head_list
ticker_head_summary_dataframe['short_rr'] = float('NaN')
ticker_head_summary_dataframe['long_rr'] = float('NaN')
ticker_head_summary_dataframe['short_sum'] = float('NaN')
ticker_head_summary_dataframe['long_sum'] = float('NaN')
ticker_head_summary_dataframe['short_r1'] = float('NaN')
ticker_head_summary_dataframe['short_z1'] = float('NaN')
ticker_head_summary_dataframe['long_r1'] = float('NaN')
ticker_head_summary_dataframe['long_z1'] = float('NaN')


for i in range(len(ticker_head_list)):
    
    selected_short_trades = big_data[(big_data['Q']>85) & (big_data['QF']>60) &(big_data['tickerHead']==ticker_head_list[i])]
    selected_long_trades = big_data[(big_data['Q']<15) & (big_data['QF']<40) &(big_data['tickerHead']==ticker_head_list[i])]
    
    
    standard_short_stats = bs.get_summary_stats(selected_short_trades['pnl_final'].values)
    standard_long_stats = bs.get_summary_stats(selected_long_trades['pnl_final'].values)
    
    
    
    ticker_head_summary_dataframe['short_rr'][i] = standard_short_stats['reward_risk']
    ticker_head_summary_dataframe['long_rr'][i] = standard_long_stats['reward_risk']
    
    ticker_head_summary_dataframe['short_sum'][i] = standard_short_stats['total_pnl']
    ticker_head_summary_dataframe['long_sum'][i] = standard_long_stats['total_pnl']
    
    ticker_head_summary_dataframe['short_r1'][i] = selected_short_trades['r1'].mean()
    ticker_head_summary_dataframe['short_z1'][i] = selected_short_trades['z1'].mean()
    
    ticker_head_summary_dataframe['long_r1'][i] = selected_long_trades['r1'].mean()
    ticker_head_summary_dataframe['long_z1'][i] = selected_long_trades['z1'].mean()
    
    
ticker_head_summary_dataframe

In [ ]:
# Ticker Head Based Analysis

import contract_utilities.contract_meta_info as cmi
ticker_head_list = cmi.futures_butterfly_strategy_tickerhead_list
import pandas as pd
import backtesting.backtest_stats as bs

ticker_head_summary_dataframe = pd.DataFrame()
ticker_head_summary_dataframe['ticker_head'] = ticker_head_list
ticker_head_summary_dataframe['short_rr'] = float('NaN')
ticker_head_summary_dataframe['long_rr'] = float('NaN')
ticker_head_summary_dataframe['short_sum'] = float('NaN')
ticker_head_summary_dataframe['long_sum'] = float('NaN')
ticker_head_summary_dataframe['short_r1'] = float('NaN')
ticker_head_summary_dataframe['short_z1'] = float('NaN')
ticker_head_summary_dataframe['long_r1'] = float('NaN')
ticker_head_summary_dataframe['long_z1'] = float('NaN')


for i in range(len(ticker_head_list)):
    
    selected_short_trades = big_data[(big_data['Q']>85) & 
                                     (big_data['QF']>60) &
                                     (big_data['r2']>50)&
                                     (big_data['z2']>1.5) &
                                     (big_data['tickerHead']==ticker_head_list[i])]
    selected_long_trades = big_data[(big_data['Q']<15) & 
                                    (big_data['QF']<40) &
                                     (big_data['r2']>50) &
                                     (big_data['z2']<-1.5) &
                                    (big_data['tickerHead']==ticker_head_list[i])]
    
    
    standard_short_stats = bs.get_summary_stats(selected_short_trades['pnl_2'].values)
    standard_long_stats = bs.get_summary_stats(selected_long_trades['pnl_2'].values)
    
    
    
    ticker_head_summary_dataframe['short_rr'][i] = standard_short_stats['reward_risk']
    ticker_head_summary_dataframe['long_rr'][i] = standard_long_stats['reward_risk']
    
    ticker_head_summary_dataframe['short_sum'][i] = standard_short_stats['total_pnl']
    ticker_head_summary_dataframe['long_sum'][i] = standard_long_stats['total_pnl']
    
    ticker_head_summary_dataframe['short_r1'][i] = selected_short_trades['r1'].mean()
    ticker_head_summary_dataframe['short_z1'][i] = selected_short_trades['z1'].mean()
    
    ticker_head_summary_dataframe['long_r1'][i] = selected_long_trades['r1'].mean()
    ticker_head_summary_dataframe['long_z1'][i] = selected_long_trades['z1'].mean()
    
    
ticker_head_summary_dataframe

In [ ]:
# Measuring Risk

import matplotlib.pyplot as plt
plt.scatter(big_data['recent_vol_ratio'],big_data['pnl_short5'])
plt.grid()
plt.show()





In [ ]:
import sys
sys.path.append(r'C:\Users\kocat_000\quantFinance\PycharmProjects')
import backtesting.backtest_futures_butterfly as bfb
import pandas as pd
import shared.directory_names as dn

#portfolio_output2 = bfb.construct_futures_butterfly_portfolio(date_list = selected_dates)

portfolio_output1 = pd.read_pickle(dn.backtest_results_folder + '/futures_butterfly/portfolio.pkl')


import matplotlib.pyplot as plt
plt.plot(range(len(portfolio_output2.index)),portfolio_output2['portfolio'].cumsum(),
        range(len(portfolio_output2.index)),portfolio_output1['portfolio'].cumsum())
plt.grid()
plt.show()

In [ ]:
import backtesting.backtest_futures_butterfly as bfb
#wuhu = bfb.construct_futures_butterfly_portfolio(rule_no=2,date_list=selected_dates)
wuhu.loc[wuhu['pnl_short1'] < -1000,'pnl_final']=wuhu.loc[wuhu['pnl_short1'] < -1000,'pnl_short2']
wuhu
wuhu.loc[wuhu['pnl_short1'] < -1000]['pnl_short2']
wuhu.loc[wuhu['pnl_short1'] < -1000]['pnl_final']

In [ ]:

import backtesting.backtest_futures_butterfly as bfb
import contract_utilities.contract_meta_info as cmi
import numpy as np
import pandas as pd
ticker_head_list = cmi.futures_butterfly_strategy_tickerhead_list



contributors_output1 = bfb.analyze_portfolio_contributors(date_list=selected_dates,date_from = 20000101,date_to = 20160101,rule_no=1)
contributors_output2 = bfb.analyze_portfolio_contributors(date_list=selected_dates,date_from = 20000101,date_to = 20160101,rule_no=2)






In [ ]:
import matplotlib.pyplot as plt
total_pnl1 = contributors_output1['total_pnl_frame']
total_pnl2 = contributors_output2['total_pnl_frame']
plt.plot(range(len(total_pnl1.index)),total_pnl1['portfolio'].cumsum(),
         range(len(total_pnl1.index)),total_pnl2['portfolio'].cumsum())
plt.legend(['no stop','stop'],frameon=False)
plt.grid()
plt.show()

In [ ]:
total_pnl1[total_pnl1['portfolio']<-200000]

sheet = backtest_output[58]

ho_sheet = sheet[sheet['tickerHead']=='HO']
ho_sheet.loc[(ho_sheet['Q']<=15)&(ho_sheet['QF']<=40)]


In [ ]:
import get_price.get_futures_price as gfp
futures_data = gfp.get_futures_price_4ticker(ticker='HOQ2007')

import matplotlib.pyplot as plt
plt.plot(futures_data['close_price'])
plt.grid()
plt.show()



In [ ]:
import backtesting.backtest_stats as bs
portfolio_stats1 = bs.get_summary_stats(total_pnl1['portfolio'])
portfolio_stats2 = bs.get_summary_stats(total_pnl2['portfolio'])

summary_table = pd.DataFrame({'class': ['Portfolio1', 'Portfolio2'],
              'reward_risk': [portfolio_stats1['reward_risk'],
                              portfolio_stats2['reward_risk']],
              'mean_pnl': [portfolio_stats1['mean_pnl'],
                           portfolio_stats2['mean_pnl']],
                'downside20': [portfolio_stats1['downside20'],
                              portfolio_stats2['downside20']],   
                 'downside5': [portfolio_stats1['downside5'],
                              portfolio_stats2['downside5']],              
              'total_pnl': [portfolio_stats1['total_pnl'],
                              portfolio_stats2['total_pnl']]})

summary_table[['class','reward_risk','mean_pnl','downside20','downside5','total_pnl']]



In [ ]:
import get_price.get_futures_price as gfp
import contract_utilities.contract_meta_info as cmi
import shared.calendar_utilities as cu
import opportunity_constructs.utilities as opUtil
import contract_utilities.expiration as exp

report_date=20120711
post_report_date = exp.doubledate_shift_bus_days(double_date=report_date,shift_in_days=-20)

futures_data_dictionary = {x: gfp.get_futures_price_preloaded(ticker_head=x) for x in cmi.futures_butterfly_strategy_tickerhead_list}


butterflies = bfb.backtest_futures_butterfly_4date(report_date=report_date,futures_data_dictionary=futures_data_dictionary )

ng_butterflies = butterflies[butterflies['tickerHead']=='S']
ng_short = ng_butterflies[(ng_butterflies['Q']<15) & (ng_butterflies['QF'] < 40)]       

ng_short.columns

ng_short[['ticker1','ticker2','ticker3','r1','r2','z1','z2','Q','QF','pnl_2']]





In [ ]:
import sys
sys.path.append(r'C:\Users\kocat_000\quantFinance\PycharmProjects')
import signals.futures_signals as fs
import shared.statistics as stats
strategy_out = fs.get_futures_butterfly_signals({'ticker_list': ['NGV2016', 'NGX2016', 'NGF2017'],'date_to': 20151104})
strategy_out['noise_ratio']

In [1]:
import futures_charts.futures_curve as fc
#fc.get_futures_curve_chart_4date(ticker_head='HO',settle_date=20070214)
aligned_output = fc.get_butterfly_panel_plot(report_date=20070214,id=124 )
#fc.get_butterfly_scatter_plot(report_date=20070214,id=124)
#wuhu['c1']['close_price'][0]-2*wuhu['c2']['close_price'][0]+wuhu['c3']['close_price'][0]

In [8]:

import matplotlib.pyplot as plt
plt.plot(aligned_data['settle_date'],
         aligned_data['c1']['close_price'])
plt.grid()
plt.show()

In [ ]:
# Basic Stats:

import math as m
import numpy as np

total_pnl = [x['total_pnl'] for x in backtest_output]
print('daily sd :' + str(round(np.std(total_pnl)/m.sqrt(5))))
print('daily mean :' + str(round(np.mean(total_pnl)/5)))
print('annualized sharp :' + str(m.sqrt(52)*np.mean(total_pnl)/np.std(total_pnl)))




In [ ]:
portfolio_pnl = [x['total_pnl'] for x in backtest_output]


import matplotlib.pyplot as plt
plt.plot(np.cumsum(portfolio_pnl))
plt.grid()
plt.show()

In [ ]:

# Tickerhead Based Results:

import contract_utilities.contract_meta_info as cmi
ticker_head_list = cmi.futures_butterfly_strategy_tickerhead_list
pnl_ticker_head_dictionary = dict()

for ticker_head in ticker_head_list:
    
    pnl_path_per_ticker_head = []
    
    for backtest_result in backtest_output:
        ticker_head_result = backtest_result['pnl_per_tickerhead'][backtest_result['pnl_per_tickerhead']['tickerHead']==ticker_head]
        pnl_path_per_ticker_head.append(ticker_head_result['mean_long'].sum()+ticker_head_result['mean_short'].sum())
    
    pnl_ticker_head_dictionary[ticker_head] = pnl_path_per_ticker_head
  


In [ ]:
# Tickerhead Based Stats

import shared.statistics as stats
import pandas as pd
import numpy as np

ticker_head_stats = pd.DataFrame({'ticker_head': ticker_head_list})

ticker_head_stats['long_mean'] = float('NaN')
ticker_head_stats['long_rr'] = float('NaN')
ticker_head_stats['short_mean'] = float('NaN')
ticker_head_stats['short_rr'] = float('NaN')


short_trades_list = [x['short_trades'] for x in backtest_output]
long_trades_list = [x['long_trades'] for x in backtest_output]

short_trades_frame  = pd.concat(short_trades_list)
long_trades_frame  = pd.concat(long_trades_list)

long_trades_frame

for i in range(len(ticker_head_list)):
    long_pnl_tickerhead = long_trades_frame['pnl'][long_trades_frame['tickerHead'] == ticker_head_list[i]]
    short_pnl_tickerhead = short_trades_frame['pnl'][short_trades_frame['tickerHead'] == ticker_head_list[i]]
    
    if len(long_pnl_tickerhead) > 0:
        ticker_head_stats['long_mean'][i] =np.nanmean(long_pnl_tickerhead)
        long_downside = stats.get_number_from_quantile(y=long_pnl_tickerhead.values,quantile_list=[20],clean_num_obs=10)
        ticker_head_stats['long_rr'][i] = ticker_head_stats['long_mean'][i]/abs(long_downside[0])
        
        
        ticker_head_stats['short_mean'][i] =np.nanmean(short_pnl_tickerhead)
        short_downside = stats.get_number_from_quantile(y=short_pnl_tickerhead.values,quantile_list=[20],clean_num_obs=10)
        ticker_head_stats['short_rr'][i] = ticker_head_stats['short_mean'][i]/abs(short_downside[0])
    
    
    
    
    


ticker_head_stats



In [ ]:
# Impact of Z1

selected_short = short_trades_frame[short_trades_frame['z1']>(short_trades_frame['z1'].mean())]
selected_short_mean = selected_short['pnl'].mean()
selected_short_downside = stats.get_number_from_quantile(y=selected_short['pnl'].values,quantile_list=[20],clean_num_obs=10)

print('selected short mean: ' + str(selected_short_mean))
print('selected short downside: ' + str(selected_short_downside[0]))
print('selected short risk reward : ' + str(selected_short_mean/abs(selected_short_downside[0])))

selected_long = long_trades_frame[long_trades_frame['z1']>(long_trades_frame['z1'].mean())]
selected_long_mean = selected_long['pnl'].mean()
selected_long_downside = stats.get_number_from_quantile(y=selected_long['pnl'].values,quantile_list=[20],clean_num_obs=10)

print('selected long mean: ' + str(selected_long_mean))
print('selected long downside: ' + str(selected_long_downside[0]))
print('selected long risk reward : ' + str(selected_long_mean/abs(selected_long_downside[0])))

# Results are Encouraging



In [ ]:
# Impact of R1

selected_short = short_trades_frame[short_trades_frame['r1']<(short_trades_frame['r1'].mean())]
selected_short_mean = selected_short['pnl'].mean()
selected_short_downside = stats.get_number_from_quantile(y=selected_short['pnl'].values,quantile_list=[20],clean_num_obs=10)

print('selected short mean: ' + str(selected_short_mean))
print('selected short downside: ' + str(selected_short_downside[0]))
print('selected short risk reward : ' + str(selected_short_mean/abs(selected_short_downside[0])))

selected_long = long_trades_frame[long_trades_frame['r1']<(long_trades_frame['r1'].mean())]
selected_long_mean = selected_long['pnl'].mean()
selected_long_downside = stats.get_number_from_quantile(y=selected_long['pnl'].values,quantile_list=[20],clean_num_obs=10)

print('selected long mean: ' + str(selected_long_mean))
print('selected long downside: ' + str(selected_long_downside[0]))
print('selected long risk reward : ' + str(selected_long_mean/abs(selected_long_downside[0])))

# Results are Encouraging

In [ ]:
short_trades_frame['z1'].mean()

In [ ]:
backtest_output[1]['pnl_per_tickerhead']

In [ ]:
butt = backtest_output[1]['butterflies']
butt['pnl'] = backtest_output[1]['butterfly_pnl']

ng_trades = butt[butt['tickerHead']=='ED']


short_trades = ng_trades[(ng_trades['Q']>85) & (ng_trades['QF']>60)]


short_trades[['ticker1','ticker2','ticker3','z1','z2','second_spread_weight_1','Q','QF','pnl']]


In [ ]:
short_trades.columns
